# 03: Quickstart — From Zero to Enforced Architecture

You have an existing TypeScript project with a `src/domain`, `src/application`, and `src/infrastructure` directory. It compiles fine, but nothing stops a junior developer from importing `fs` in the domain layer or calling the database directly from a controller.

This notebook walks through adding KindScript enforcement in **4 commands**:
1. `ksc init --detect` — detect your architecture
2. `ksc infer --write` — generate architecture definitions
3. `ksc check` — verify contracts (should pass)
4. Introduce a violation, `ksc check` again — see it caught

**Prerequisites:** Run `npx tsc` from the project root to compile the CLI.

## Setup: Helper and CLI path

In [ ]:
// Helper to run CLI commands and display output
const PROJECT_ROOT = Deno.cwd().replace(/\/notebooks$/, "");
const KSC = PROJECT_ROOT + "/dist/infrastructure/cli/main.js";

async function ksc(...args: string[]): Promise<{ code: number; output: string }> {
  const cmd = new Deno.Command("node", {
    args: [KSC, ...args],
    stdout: "piped",
    stderr: "piped",
  });
  const { code, stdout, stderr } = await cmd.output();
  const out = new TextDecoder().decode(stdout);
  const err = new TextDecoder().decode(stderr);
  const output = (out + err).trim();
  if (output) console.log(output);
  console.log(`\nExit code: ${code}`);
  return { code, output };
}

async function shell(command: string): Promise<string> {
  const cmd = new Deno.Command("bash", {
    args: ["-c", command],
    stdout: "piped",
    stderr: "piped",
  });
  const { stdout, stderr } = await cmd.output();
  const out = (new TextDecoder().decode(stdout) + new TextDecoder().decode(stderr)).trim();
  if (out) console.log(out);
  return out;
}

console.log("CLI path:", KSC);

## Step 0: Create a realistic project

We'll create a small Clean Architecture project with three layers:
- **Domain** — pure business logic (User entity)
- **Application** — use case orchestration (register user)
- **Infrastructure** — external adapters (in-memory repository)

In [ ]:
const DEMO = Deno.makeTempDirSync({ prefix: "ksc-quickstart-" });
console.log("Demo project:", DEMO);

// Create directory structure
Deno.mkdirSync(`${DEMO}/src/domain`, { recursive: true });
Deno.mkdirSync(`${DEMO}/src/application`, { recursive: true });
Deno.mkdirSync(`${DEMO}/src/infrastructure`, { recursive: true });

// Domain entity — pure business logic, no external dependencies
Deno.writeTextFileSync(`${DEMO}/src/domain/user.ts`, `
export interface User {
  id: string;
  name: string;
  email: string;
}

export function createUser(name: string, email: string): User {
  return { id: Math.random().toString(36), name, email };
}
`.trimStart());

// Application use case — orchestrates domain objects
Deno.writeTextFileSync(`${DEMO}/src/application/register-user.ts`, `
import { createUser, User } from '../domain/user';

export function registerUser(name: string, email: string): User {
  const user = createUser(name, email);
  console.log('Registered:', user.name);
  return user;
}
`.trimStart());

// Infrastructure adapter — database repository
Deno.writeTextFileSync(`${DEMO}/src/infrastructure/user-repo.ts`, `
import { User } from '../domain/user';

const store: User[] = [];

export function saveUser(user: User): void {
  store.push(user);
}

export function findAllUsers(): User[] {
  return [...store];
}
`.trimStart());

// tsconfig.json
Deno.writeTextFileSync(`${DEMO}/tsconfig.json`, JSON.stringify({
  compilerOptions: {
    target: "ES2020",
    module: "commonjs",
    strict: true,
    rootDir: "src",
    outDir: "dist",
  },
  include: ["src/**/*.ts"],
}, null, 2));

// Show the project structure
await shell(`find ${DEMO} -type f | sort | sed 's|${DEMO}/||'`);

## Step 1: Detect your architecture

KindScript scans your directory structure and import graph to identify the architectural pattern.

In [ ]:
await ksc("init", "--detect", DEMO);

KindScript detected **Clean Architecture** with three layers and their dependency directions. The `init --detect` command generates TypeScript project references (`tsconfig.json` per layer) for zero-config boundary enforcement.

But we want more than project references — we want full contract enforcement.

## Step 2: Generate architecture definitions

`ksc infer` generates `architecture.ts` — the source of truth for your architectural rules. It contains:
- **Kind definitions** — TypeScript interfaces for each layer
- **Instance declaration** — maps kinds to actual directories
- **Contracts** — rules like "domain must not depend on infrastructure"

In [ ]:
await ksc("infer", "--write", DEMO);

In [ ]:
// Let's see what was generated
console.log("=== architecture.ts ===");
console.log(Deno.readTextFileSync(`${DEMO}/architecture.ts`));
console.log("=== kindscript.json ===");
console.log(Deno.readTextFileSync(`${DEMO}/kindscript.json`));

Two files were generated:
- **`architecture.ts`** — Kind definitions, instance mapping, and contracts. This is a real TypeScript file you check into source control.
- **`kindscript.json`** — points to the definitions file. Analogous to `tsconfig.json`.

The inferred contracts say:
- `noDependency`: domain cannot import from application or infrastructure; application cannot import from infrastructure
- `purity`: all layers must be free of Node.js I/O imports

## Step 3: Check contracts

Our project follows the rules — the check should pass.

In [ ]:
await ksc("check", DEMO);

All contracts satisfied. The architecture is enforced.

## Step 4: Break a rule and see it caught

Now let's introduce a violation. The domain layer should never depend on infrastructure — but what if someone adds a direct database import to the domain entity?

In [ ]:
// Introduce a violation: domain imports from infrastructure
Deno.writeTextFileSync(`${DEMO}/src/domain/user.ts`, `
import { findAllUsers } from '../infrastructure/user-repo';

export interface User {
  id: string;
  name: string;
  email: string;
}

export function createUser(name: string, email: string): User {
  // BAD: domain reaching into infrastructure
  const existing = findAllUsers();
  console.log(existing.length + ' users already exist');
  return { id: Math.random().toString(36), name, email };
}
`.trimStart());

console.log("Violation introduced: domain/user.ts now imports from infrastructure/user-repo");

In [ ]:
// Run ksc check — should fail with KS70001
await ksc("check", DEMO);

**KindScript caught it.** The error tells you:
- **Where**: `src/domain/user.ts` line 1
- **What**: Forbidden dependency — domain is importing from infrastructure
- **Why**: The `noDependency(domain -> infrastructure)` contract
- **Error code**: `KS70001` (same format as TypeScript errors)

The exit code is `1`, so CI pipelines will fail automatically.

## Step 5: Fix and verify

Remove the bad import. The domain layer should depend only on its own types.

In [ ]:
// Fix: remove the infrastructure import
Deno.writeTextFileSync(`${DEMO}/src/domain/user.ts`, `
export interface User {
  id: string;
  name: string;
  email: string;
}

export function createUser(name: string, email: string): User {
  return { id: Math.random().toString(36), name, email };
}
`.trimStart());

await ksc("check", DEMO);

## Summary

In 4 commands, you went from an unguarded project to enforced architectural boundaries:

| Command | What it does |
|---------|-------------|
| `ksc init --detect` | Scans project, detects architectural pattern |
| `ksc infer --write` | Generates `architecture.ts` with kinds, instances, and contracts |
| `ksc check` | Validates all contracts, exit code 0 if clean, 1 if violations |

The generated `architecture.ts` is a real TypeScript file — you edit it, add contracts, and check it into source control. It becomes the single source of truth for your architectural rules.

**Next steps:**
- See **04-all-contract-types.ipynb** for all 5 contract types
- See **05-architecture-inference.ipynb** for scaffolding new projects
- See **07-ci-integration.ipynb** for CI/CD setup

In [ ]:
// Cleanup
Deno.removeSync(DEMO, { recursive: true });
console.log("Demo project cleaned up.");